In [1]:
import pandas as pd

In [39]:
import xgboost as xgb

In [24]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [10]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,Rear,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,Rear,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,Front,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,Rear,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,Rear,208,0.93,54,58,Rear


In [25]:
test.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,2c9180975a056a64015a1e10d3f270fe,Right,67,0.63,107,169
1,2c9180975a056a64015a1de4deb16bdc,Front,16,0.88,61,69
2,2c9180975a056a64015a1e0e70ea70ce,Right,44,1.15,232,202
3,2c9180975a056a64015a1dfed0c46ec6,Right,50,1.10,137,125
4,2c9180975a056a64015a1dfed0c46ec7,Front,30,0.95,99,104


In [14]:
features = train[['DetectedCamera', 'AngleOfSign', 'SignAspectRatio', 'SignWidth', 'SignHeight']]

In [26]:
train['DetectedCamera'].value_counts()

Front    10910
Right    10516
Left      9298
Rear      7761
Name: DetectedCamera, dtype: int64

In [27]:
#encode as integer
mapping = {'Front':0, 'Right':1, 'Left':2, 'Rear':3}
train = train.replace({'DetectedCamera':mapping})
test = test.replace({'DetectedCamera':mapping})

In [28]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,2c9180975a056a64015a1e0a52e57021,3,195,1.02,46,45,Rear
1,2c9180975a056a64015a1e17b32171e4,3,203,1.09,59,54,Rear
2,2c9180975a056a64015a1de4deb16bd5,0,26,0.96,104,108,Front
3,2c9180975a056a64015a1de4deb16bdd,3,199,0.81,38,47,Rear
4,2c9180975a056a64015a1de4deb16bd6,3,208,0.93,54,58,Rear


In [29]:
#renaming column
train.rename(columns = {'SignFacing (Target)': 'Target'}, inplace=True)

In [30]:
#encode Target Variable based on sample submission file
mapping = {'Front':0, 'Left':1, 'Rear':2, 'Right':3}
train = train.replace({'Target':mapping})

In [33]:
train.head()

,Id,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,Target
0,2c9180975a056a64015a1e0a52e57021,3,195,1.02,46,45,2
1,2c9180975a056a64015a1e17b32171e4,3,203,1.09,59,54,2
2,2c9180975a056a64015a1de4deb16bd5,0,26,0.96,104,108,0
3,2c9180975a056a64015a1de4deb16bdd,3,199,0.81,38,47,2
4,2c9180975a056a64015a1de4deb16bd6,3,208,0.93,54,58,2


In [31]:
train.columns.values

array(['Id', 'DetectedCamera', 'AngleOfSign', 'SignAspectRatio',
       'SignWidth', 'SignHeight', 'Target'], dtype=object)

In [32]:
for col in train.columns.values:
    print train[col].dtype

object
int64
int64
float64
int64
int64
int64


In [34]:
#target variable
y_train = train['Target']
test_id = test['Id']

In [35]:

#drop columns
train.drop(['Target','Id'], inplace=True, axis=1)
test.drop('Id',inplace=True,axis=1)

In [36]:
train.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,3,195,1.02,46,45
1,3,203,1.09,59,54
2,0,26,0.96,104,108
3,3,199,0.81,38,47
4,3,208,0.93,54,58


In [37]:
y_train.head()

0    2
1    2
2    0
3    2
4    2
Name: Target, dtype: int64

In [45]:
dtrain = xgb.DMatrix(train, y_train)
dtest = xgb.DMatrix(test)

In [46]:
dtrain.get_label()

array([ 2.,  2.,  0., ...,  2.,  2.,  0.], dtype=float32)

In [50]:
from xgboost.sklearn import XGBClassifier

In [51]:
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

# Tensorflow Model

In [59]:
train_new = pd.read_csv("train.csv")
test_new = pd.read_csv("test.csv")

In [62]:
test_id_new = test_new['Id']

In [63]:

#drop columns
train_new.drop(['Id'], inplace=True, axis=1)
test_new.drop('Id',inplace=True,axis=1)

In [64]:
train_new.head()

,DetectedCamera,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,SignFacing (Target)
0,Rear,195,1.02,46,45,Rear
1,Rear,203,1.09,59,54,Rear
2,Front,26,0.96,104,108,Front
3,Rear,199,0.81,38,47,Rear
4,Rear,208,0.93,54,58,Rear


In [65]:
train_new_dummy = pd.get_dummies(train_new)
test_new_dummy = pd.get_dummies(test_new)

In [66]:
train_new_dummy.head()

,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,DetectedCamera_Front,DetectedCamera_Left,DetectedCamera_Rear,DetectedCamera_Right,SignFacing (Target)_Front,SignFacing (Target)_Left,SignFacing (Target)_Rear,SignFacing (Target)_Right
0,195,1.02,46,45,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,203,1.09,59,54,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,26,0.96,104,108,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,199,0.81,38,47,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,208,0.93,54,58,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [69]:
#renaming column
train_new_dummy.rename(columns = {'SignFacing (Target)_Front': 'Front', 'SignFacing (Target)_Left': 'Left', 'SignFacing (Target)_Rear': 'Rear', 'SignFacing (Target)_Right': 'Right'}, inplace=True)

In [70]:
train_new_dummy.head()

,AngleOfSign,SignAspectRatio,SignWidth,SignHeight,DetectedCamera_Front,DetectedCamera_Left,DetectedCamera_Rear,DetectedCamera_Right,Front,Left,Rear,Right
0,195,1.02,46,45,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,203,1.09,59,54,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,26,0.96,104,108,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,199,0.81,38,47,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,208,0.93,54,58,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0


In [74]:
features = train_new_dummy[['DetectedCamera_Front', 'DetectedCamera_Left', 'DetectedCamera_Rear', 'DetectedCamera_Right', 'AngleOfSign', 'SignAspectRatio', 'SignWidth', 'SignHeight']]
targets = train_new_dummy[['Front', 'Left', 'Rear', 'Right']]

In [75]:
features.head()

,DetectedCamera_Front,DetectedCamera_Left,DetectedCamera_Rear,DetectedCamera_Right,AngleOfSign,SignAspectRatio,SignWidth,SignHeight
0,0.0,0.0,1.0,0.0,195,1.02,46,45
1,0.0,0.0,1.0,0.0,203,1.09,59,54
2,1.0,0.0,0.0,0.0,26,0.96,104,108
3,0.0,0.0,1.0,0.0,199,0.81,38,47
4,0.0,0.0,1.0,0.0,208,0.93,54,58


In [90]:
features.to_csv("features.csv")

In [77]:
targets.head()

,Front,Left,Rear,Right
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0
4,0.0,0.0,1.0,0.0


In [91]:
targets.to_csv("targets.csv")

In [56]:
import tensorflow as tf
sess = tf.InteractiveSession()

In [78]:
x = tf.placeholder(tf.float32, shape=[None, 8])
y_ = tf.placeholder(tf.float32, shape=[None, 4])

In [82]:
sess.run(tf.global_variables_initializer())

In [ ]:
filename_queue = tf.train.string_input_producer(["train.csv"])

In [93]:
reader = tf.TextLineReader()

In [108]:
key, value = reader.read(filename_queue)

In [99]:
record_defaults = [[1], [1], [1], [1], [1], [1], [1], [1]]

In [103]:
col1, col2, col3, col4, col5, col6, col7, col8 = tf.decode_csv(
    key, record_defaults=record_defaults)

In [106]:
col9, col10, col11, col12 = tf.decode_csv(
    value, record_defaults=[[1], [1], [1], [1]])

In [110]:
node1 = tf.constant(3.0)

In [79]:
W = tf.Variable(tf.zeros([8,4]))
b = tf.Variable(tf.zeros([4]))

In [80]:
y = tf.matmul(x,W) + b

In [81]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [83]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

In [ ]:
for _ in range(1000):
  batch = train.next_batch(100)
  train_step.run(feed_dict={x: batch[0], y_: batch[1]})

# Another Approach